# 1D Mirror Detection Learner
**Goal**: correctly identify whether an abstract mirror object is within the vision range

In [1]:
import tensorflow as tf
import numpy

## Define constants

In [2]:
image_width = 28
image_height = 28
vision_range = image_width * image_height
category_count = 10
label_type_count = 2  # mirror or not
sample_count = {
    "training": 80,
    "test": 13,
    "validation": 7
    }
learning_rate = 0.5

## Auxiliary functions

### Show result

In [3]:
def show_result(x):
    """Print the result"""
    print("{:.0f}%".format(x * 100))

### Automaticaly generate data sets

In [4]:
def random_mirror(vision_range, category_count):
    """Return a n-array with either a randomly placed mirror
        or no mirror at all.
    
    Args:
        vision_range (int): how many objects to be seen at a time
        category_count (int): total number of different objects
            including the mirror type
    Return:
        2-tuple:
             [0]: an array of size same as the "vision_range".
             [1]: 1 one-hot 2-array [1, 0] (has mirror) or [0, 1] (no mirror).
    """
    def label(x):
        if x is True:
            return [1, 0]
        else:
            return [0, 1]
    p1 = numpy.random.randint(1, high=category_count, size=vision_range-1)
    p2 = numpy.random.randint(0, category_count, 1)
    data = numpy.random.permutation(numpy.concatenate((p1, p2)))
    return (data, label(0 in data))



In [5]:
def mirror_data(n, vision_range, category_count):
    """Return a n x size matrix
    Args:
        n (int): number of data points
        vision_range (int): number of objects to be seen at a time
        category_count (int): total number of different object categories
            including the mirror
    Returns:
        A dictionary:
            data: (n, v)-sized 2D numpy array where v is the vision_range
            labels: (n,)-sized 1D numpy array (each element is either 1 or 0)
    """
    raw = [random_mirror(vision_range, category_count) for i in range(n)]
    return {
        "data": numpy.array([x[0] for x in raw]),
        "labels": numpy.array([x[1] for x in raw])
    }

## Dataset

In [6]:
dataset = {
    "training": mirror_data(sample_count['training'], vision_range, category_count),
    "test": mirror_data(sample_count['test'], vision_range, category_count),
    "validation": mirror_data(sample_count['validation'], vision_range, category_count)
}

# Start a TensorFlow session

In [7]:
session = tf.InteractiveSession()

# Memory allocation

In [8]:
W = tf.Variable(tf.zeros([vision_range, label_type_count]))

In [9]:
b = tf.Variable(tf.zeros([label_type_count]))

In [10]:
x = tf.placeholder(tf.float32, [None, vision_range])

In [11]:
y_ = tf.placeholder(tf.float32, [None, label_type_count])

In [12]:
session.run(tf.global_variables_initializer())

# Predictions and losses

In [13]:
y = tf.matmul(x, W) + b

# Convolutionary Neural Net

## Weight Initialization

In [14]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


In [15]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [16]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

In [17]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1],
                         padding="SAME")

In [18]:
W_conv1 = weight_variable([5, 5, 1, 32])

In [19]:
b_conv1 = bias_variable([32])

In [20]:
x_image = tf.reshape(x, [-1, image_width, image_height, 1])

In [21]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) +  b_conv1)

In [22]:
h_pool1 = max_pool_2x2(h_conv1)

## Second Convolutional Layer

In [23]:
W_conv2 = weight_variable([5, 5, 32, 64])

In [24]:
b_conv2 = bias_variable([64])

In [25]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) +  b_conv2)

In [26]:
h_pool2 = max_pool_2x2(h_conv2)

## Densely connected layer

In [27]:
W_fc1 = weight_variable([7 * 7 *64, 1024])
b_fc1 = bias_variable([1024])

In [28]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 *64])

In [29]:
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## Dropout

In [30]:
keep_prob = tf.placeholder(tf.float32)

In [31]:
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## Readout layer

In [32]:
W_fc2 = weight_variable([1024, label_type_count])

In [33]:
b_fc2 = bias_variable([label_type_count])

In [34]:
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

## Training and evaluation

In [35]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))

In [36]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [37]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

In [38]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [39]:
session.run(tf.global_variables_initializer())

In [40]:
train_accuracy = accuracy.eval(
    feed_dict={x: dataset['training']['data'], y_: dataset['training']['labels'], keep_prob: 1.0})

In [41]:
train_step.run(feed_dict={x: dataset['training']['data'], y_: dataset['training']['labels'], 
                         keep_prob: 1.0})

# Test

In [42]:
test_accuracy = accuracy.eval(feed_dict={
        x: dataset['test']['data'],
        y_: dataset['test']['labels'],
        keep_prob: 1.0
    })

In [43]:
show_result(test_accuracy)

92%


# Validation

In [ ]:
validation_accuracy = accuracy.eval(
    feed_dict={
        x: dataset['validatioin']['data'],
        y_: dataset['validation']['labels'],
        keep
    }
)